<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_6)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проверяем версию компилятора NVIDIA CUDA

In [ ]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 66 kB 4.1 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627577 sha256=fab00ebbd5cf1bcb74c709ea1a5bb86757d552620115240bde58665ee6ec4024
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=8512d3fc973712b58d8a95599d3b6daf2f32ed6fa06fc7a1047c9081c1bf7b51
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


**Using the PyCUDA module**

Для того, чтобы инициализировать CUDA драйвер, необходимо сменить тип Runtime'a в Google Collab на тот, который поддерживает аппаратное ускорение GPU. 
Для этого необходимо: 

*   Нажать на вкладку `Runtime` и выбрать `Change runtime type`.
*   После этого в разделе `Hardware Acceleration`, выбрать `GPU` и нажать `Save`.




In [ ]:
import pycuda.driver as drv 
drv.init() 
print("%d device(s) found." % drv.Device.count()) 
for ordinal in range(drv.Device.count()): 
  dev = drv.Device(ordinal) 
  print("Device #%d: %s" % (ordinal, dev.name())) 
  print("Compute Capability: %d.%d" % dev.compute_capability())     
  print("Total Memory: %s KB" % (dev.total_memory()//(1024)))

1 device(s) found.
Device #0: Tesla K80
Compute Capability: 3.7
Total Memory: 11715776 KB


После этого, должно инициализироваться видеоустройство Tesla K80

**How to build a PyCUDA application**

Вычисления могут производиться как на ЦПУ, так и на ГПУ. Как правило, с помощью на ЦПУ происходит подготовка данных, после чего данные передаются на ГПУ для дальнейших трудозатратных вычислений. Обработанные результирующие данные передаются обратно на ЦПУ, для их вывода. Одно из обязательных условий является выделения памяти на ГПУ заранее перед непосредственым выполнением вычислений. 

In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

a = numpy.random.randn(5,5)
a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
__global__ void doubleMatrix(float *a)
{
  int idx = threadIdx.x + threadIdx.y*4;
a[idx] *= 2; }
""")

func = mod.get_function("doubleMatrix")
func(a_gpu, block=(5,5,1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print("ORIGINAL MATRIX")
print(a)
print("DOUBLED MATRIX AFTER PyCUDA EXECUTION")
print(a_doubled)

ORIGINAL MATRIX
[[ 0.11767171  0.1937645  -1.0480621  -0.38260517 -0.45635247]
 [-0.30242795  0.6900911   0.76791215 -0.39162463 -0.5633676 ]
 [-0.91431695  0.48246655 -1.0379905   1.0718129  -1.7166617 ]
 [ 0.32136446 -1.1683393   0.7952338  -0.7712385   1.5996156 ]
 [ 0.75574934 -0.9920836   0.05320557  1.942601    2.012559  ]]
DOUBLED MATRIX AFTER PyCUDA EXECUTION
[[ 0.23534341  0.387529   -2.0961242  -0.76521033 -0.91270494]
 [-0.6048559   1.3801821   1.5358243  -0.78324926 -1.1267352 ]
 [-1.8286339   0.9649331  -2.075981    2.1436257  -3.4333234 ]
 [ 0.6427289  -2.3366785   1.5904676  -1.542477    3.1992311 ]
 [ 1.5114987  -0.9920836   0.05320557  1.942601    2.012559  ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).



**Understanding the PyCUDA memory model
with matrix manipulation**

Не вся память одинакова по показателям скорости доступа в модели памяти ГПУ, но лучшей практикой является использование каждого
типа памяти наиболее эффективным образом. Основная идея состоит в том, чтобы свести к минимуму  доступ к глобальной памяти (global memory) за счет использования
общей памяти(shared memory). 

Этот метод обычно используется для разделения исходных данных
таким образом, чтобы мы позволяли блоку потоков выполнять свои вычисления в закрытом
подмножестве данных. Таким образом, потоки, присоединяющиеся к соответствующему блоку память, будут работать вместе, чтобы разгрузить глобальную память, и максимально утилизировать наиболее быстродейственную общую память.

In [9]:

import numpy as np
from pycuda import driver, compiler, gpuarray, tools
from pycuda.compiler import SourceModule

import pycuda.autoinit

kernel_code_template = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    float Pvalue = 0;
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * %(MATRIX_SIZE)s + tx];
        Pvalue += Aelement * Belement;
    }

    c[ty * %(MATRIX_SIZE)s + tx] = Pvalue;
}
"""

MATRIX_SIZE = 5

a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

c_cpu = np.dot(a_cpu, b_cpu)

a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE 
}

mod = compiler.SourceModule(kernel_code)

matrixmul = mod.get_function("MatrixMulKernel")

matrixmul(
    a_gpu, b_gpu, 
    c_gpu, 
    block = (MATRIX_SIZE, MATRIX_SIZE, 1),
)

# print the results
print("-" * 80)
print("Matrix A (GPU):")
print(a_gpu.get())

print("-" * 80)
print("Matrix B (GPU):")
print(b_gpu.get())

print("-" * 80)
print("Matrix C (GPU):")
print(c_gpu.get())

print("-" * 80)
print("CPU-GPU difference:")
print(c_cpu - c_gpu.get())

np.allclose(c_cpu, c_gpu.get())


--------------------------------------------------------------------------------
Matrix A (GPU):
[[-1.6566936   0.56398535  1.9507375   0.58440423 -0.7316022 ]
 [ 1.7494041  -0.5206335   0.5521091  -1.5025228  -0.6369413 ]
 [ 0.28192496  0.77173406  0.56775284 -0.1784921  -0.35580882]
 [ 0.7989966   0.2856385  -0.8577809  -2.0350919  -0.24192981]
 [ 1.4297949   0.6130067  -0.17448708 -0.01680106  0.17822081]]
--------------------------------------------------------------------------------
Matrix B (GPU):
[[ 0.62587035  1.5087065   0.8595771  -0.6473098   0.65500885]
 [ 1.9399728   2.0890336  -0.5696015   1.1675851  -0.98925865]
 [ 1.0712496   1.119891    0.35546783 -0.33462828  1.8551363 ]
 [-0.28716367  1.6689091   0.45135853  1.4509321  -0.7902891 ]
 [ 0.5076282   0.9004284  -1.5363557  -0.74062985 -0.5335114 ]]
--------------------------------------------------------------------------------
Matrix C (GPU):
[[ 1.6077662   1.1798956   0.33589873  2.4679003   1.9042774 ]
 [ 0.7844725  

True

**Kernel invocations with GPUArray**

В библиотеке PyCUDA представлен класс pycuda.gpuarray.GPUArray который предоставляет высокоуровневый интервейс для выполнения вычислений с помощью CUDA (в отличии от pycuda.compiler.SourceModule, котороый вызывается компилятором nvcc из исходного кода СUDA)

In [13]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy

a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
a_doubled = (2*a_gpu).get()
print("Doubled matrix using gpuarray call:\n", a_doubled)
print("Original matrix:\n",a_gpu)

Doubled matrix using gpuarray call:
 [[-0.76718646 -1.6005294  -1.2835188   1.3084855 ]
 [-0.7536936   1.927716   -4.4484     -0.49752158]
 [ 1.0747708  -1.7455909   1.684047    0.23103906]
 [ 1.2383196   3.658996   -2.9613283   1.3125074 ]]
Original matrix:
 [[-0.38359323 -0.8002647  -0.6417594   0.65424275]
 [-0.3768468   0.963858   -2.2242     -0.24876079]
 [ 0.5373854  -0.87279546  0.8420235   0.11551953]
 [ 0.6191598   1.829498   -1.4806641   0.6562537 ]]


**Evaluating element-wise expressions with
PyCUDA**

Функция PyCuda.elementwise.ElementwiseKernel похволяет нам выполнять сложные вычисления которые состоят из одного или более операндов за один вычислительный шаг.

В данном примере проводилось вычисления линейной комбинации двух случайно сгенерированных векторов (через pycuda.curandom) с помощью функции ElementwiseKernel. Входными параметрами данной функции стали: список аргументов, используемых для вычисления линейной комбинации 2х векторов, непосредственно определение вычисляемой линейной комбинации, а также наименование данной операции) 

In [14]:
import pycuda.autoinit
import numpy
from pycuda.curandom import rand as curand
from pycuda.elementwise import ElementwiseKernel
import numpy.linalg as la

input_vector_a = curand((50,))
input_vector_b = curand((50,))
mult_coefficient_a = 2
mult_coefficient_b = 5

linear_combination = ElementwiseKernel(
    "float a, float *x, float b, float *y, float *c",
    "c[i] = a*x[i] + b*y[i]",
    "linear_combination")

linear_combination_result = gpuarray.empty_like(input_vector_a)
linear_combination(mult_coefficient_a,
                   input_vector_a,\
                   mult_coefficient_b, input_vector_b,\
                   linear_combination_result)

print("INPUT VECTOR A =")
print(input_vector_a)

print("INPUT VECTOR B = ")
print(input_vector_b)

print("RESULTING VECTOR C = ")
print(linear_combination_result)

print("CHECKING THE RESULT EVALUATING THE DIFFERENCE VECTOR BETWEEN CAND THE LINEAR COMBINATION OF A AND B")
print("C - (%sA + %sB) = "%(mult_coefficient_a,mult_coefficient_b))

print(linear_combination_result -
      (mult_coefficient_a * input_vector_a + mult_coefficient_b * input_vector_b))

assert la.norm((linear_combination_result - \
                (mult_coefficient_a*input_vector_a +\
                 mult_coefficient_b*input_vector_b)).get()) < 1e-5

/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,
/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,


INPUT VECTOR A =
[0.16397922 0.2542899  0.1561198  0.7423383  0.48986718 0.24225038
 0.5527745  0.928087   0.9870533  0.42109317 0.32900068 0.32025725
 0.1596376  0.41115913 0.10089948 0.5432142  0.616877   0.17013712
 0.10183815 0.24852332 0.6760686  0.98465866 0.29470316 0.45642775
 0.24516201 0.5604631  0.6643754  0.07468423 0.7439757  0.315316
 0.40665123 0.52793115 0.7565336  0.20500064 0.2343749  0.8014761
 0.2337355  0.13296954 0.64089704 0.31119043 0.98951477 0.74160135
 0.8360364  0.56541276 0.6560929  0.8817107  0.01718229 0.5732893
 0.45194414 0.713942  ]
INPUT VECTOR B = 
[0.79965776 0.43139103 0.15423283 0.66269946 0.96225375 0.48722842
 0.05734671 0.46749467 0.3964474  0.276251   0.94695324 0.04347985
 0.99261177 0.60987836 0.28042185 0.8317375  0.19229913 0.6702712
 0.38508534 0.8897452  0.08817302 0.12089943 0.62499756 0.9237239
 0.187887   0.96184856 0.6624714  0.19917026 0.38990816 0.35291624
 0.21171579 0.36110634 0.4578505  0.6894811  0.63080394 0.80449146
 0.830141

/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,


**The MapReduce operation with PyCUDA**